In [1]:
import praw
import pandas as pd
import datetime as dt
import re
import csv

In [2]:
reddit = praw.Reddit(client_id='QZuxM_pRnbg5Mw',
                    client_secret='A7Ig0KJ3dTekgOUbz2y56RwEaZE',
                    user_agent='Mind Change Log',
                    username='loganamcnichols',
                    password='L703htw7e!')

In [21]:
def get_comments(limit=10):
    successful_comments = []
    unsuccessful_comments = []
    cmv = reddit.subreddit('changemyview')
    top_posts = cmv.top(limit=limit)
    for post in top_posts:
        author = post.author
        for j, comment in enumerate(post.comments):
            if comment.author == "DeltaBot":
                link = extract_link(comment)
                delta_comments = get_delta_comments(link)
                successful_comments += list(delta_comments.values())
            elif comment.author == author:
                continue
            elif comment in delta_comments.keys():
                continue
            else:
                unsuccessful_comments.append(comment.body)
                if j >= len(delta_comments):
                    break
    return successful_comments, unsuccessful_comments

In [4]:
def extract_link(comment):
    string = comment.body
    link_groups = re.search("(?<=\])\((.+?)\)", string)
    return link_groups.group(1)

In [19]:
def get_delta_comments(link):
    comment_dict = {}
    url = "https://reddit.com" + link
    submission = praw.models.Submission(reddit, url=url)
    comments = submission.selftext
    comment_links = re.findall('(?<=[...]])\((.+?)\)|(?<=[.."]])\((.+?)\)', comments)
    comment_links = [i + j for (i,j) in comment_links]
    comment_links = abbreviate(comment_links)
    for link in comment_links:
        url = "https://reddit.com" + link
        submission = praw.models.Submission(reddit, url=url)
        cid = submission.id
        comment_dict[cid] = submission.selftext
    return comment_dict

In [20]:
def abbreviate(links):
    short_links = []
    for link in links:
        pattern = re.search("/\w+/\w+/\w+/\w+", link)
        link = pattern.group(0)
        short_links.append(link)
    short_links = list(set(short_links))
    return short_links